### Test methods that work with accounts

In [1]:
from dexter.DB import DB, Account, Entry, Transaction, Document, Tag

import re

In [2]:
DB.open('demo')

### Match Account Names

In [3]:
Account.objects(name__contains='checking')

[<Account: <Acct assets:checking assets>>]

In [4]:
Account.objects(name__contains='food')

[<Account: <Acct expenses:food expenses>>, <Account: <Acct expenses:food:groceries expenses>>, <Account: <Acct expenses:food:restaurant expenses>>]

In [5]:
for a in DB.find_account('food'):
    print(a)

<Acct expenses:food expenses>
<Acct expenses:food:groceries expenses>
<Acct expenses:food:restaurant expenses>


In [6]:
DB.account_name_parts()

{'assets',
 'car',
 'chase',
 'checking',
 'equity',
 'expenses',
 'food',
 'groceries',
 'home',
 'income',
 'liabilities',
 'restaurant',
 'savings',
 'visa',
 'yoyodyne'}

In [7]:
DB.account_name_parts('expenses')

{'car', 'expenses', 'food', 'groceries', 'home', 'restaurant'}

In [8]:
DB.full_names()

{'equity': ['equity'],
 'assets': ['assets:checking', 'assets:savings'],
 'checking': ['assets:checking'],
 'savings': ['assets:savings'],
 'expenses': ['expenses:car',
  'expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant',
  'expenses:home'],
 'car': ['expenses:car'],
 'food': ['expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant'],
 'groceries': ['expenses:food:groceries'],
 'restaurant': ['expenses:food:restaurant'],
 'home': ['expenses:home'],
 'income': ['income:yoyodyne'],
 'yoyodyne': ['income:yoyodyne'],
 'liabilities': ['liabilities:chase:visa'],
 'chase': ['liabilities:chase:visa'],
 'visa': ['liabilities:chase:visa']}

In [9]:
DB.full_names('expenses')

{'expenses': ['expenses:car',
  'expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant',
  'expenses:home'],
 'car': ['expenses:car'],
 'food': ['expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant'],
 'groceries': ['expenses:food:groceries'],
 'restaurant': ['expenses:food:restaurant'],
 'home': ['expenses:home']}

In [10]:
DB.account_groups()

['equity',
 'assets:checking',
 'assets:savings',
 'expenses:car',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'income:yoyodyne',
 'liabilities:chase:visa']

In [11]:
DB.account_groups(['expenses'])

['expenses:car',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home']

In [12]:
DB.account_groups(['expenses:1'])

['expenses.*']

In [13]:
DB.account_groups(['expenses:2'])

['expenses:car.*', 'expenses:food.*', 'expenses:home.*']

In [14]:
DB.account_groups(['expenses:3'])

['expenses:car',
 'expenses:food',
 'expenses:food:groceries.*',
 'expenses:food:restaurant.*',
 'expenses:home']

In [15]:
DB.account_groups(['expenses:food'])

['expenses:food', 'expenses:food:groceries', 'expenses:food:restaurant']

In [16]:
DB.account_groups(['expenses:food:2'])

['expenses:food.*']

In [17]:
DB.account_groups(['expenses:food:3'])

['expenses:food', 'expenses:food:groceries.*', 'expenses:food:restaurant.*']

In [18]:
DB.account_groups(['expenses','income'])

['expenses:car',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'income:yoyodyne']

In [19]:
DB.account_groups(['expenses:1','income:1'])

['expenses.*', 'income.*']

In [20]:
DB.account_groups(['expenses:2','income:2'])

['expenses:car.*', 'expenses:food.*', 'expenses:home.*', 'income:yoyodyne.*']

### Balances

In [21]:
DB.select(Entry, account='expenses:food*')

[<Entry: <En 2024-01-02 expenses:food -$1000.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-01-07 expenses:food:restaurant +$75.0 []>>, <Entry: <En 2024-01-08 expenses:food:restaurant -$35.0 []>>]

In [22]:
for e in DB.select(Entry, account='expenses:food*'):
    print(e)

<En 2024-01-02 expenses:food -$1000.0 [<Tag.B: '#budget'>]>
<En 2024-01-07 expenses:food:restaurant +$75.0 []>
<En 2024-01-08 expenses:food:restaurant -$35.0 []>


In [28]:
for t in DB.select(Transaction, tag='#budget'):
    print(t)

<Tr 2024-01-02 expenses:home/expenses:car/expenses:food -> income:yoyodyne $5000.0 fill envelopes ['#budget']>


In [ ]:
def balance(acct, budgets=True):
    res = sum(e.value for e in DB.select(Entry, account=acct))
    if not budgets:
        res -= sum(e.value for e in DB.select(Entry, account=acct, tag=Tag.B))
    return res

In [36]:
balance('expenses:food')

-960.0

In [37]:
balance('expenses:food', False)

40.0

In [38]:
%timeit balance('expenses:food', False)

438 μs ± 6.86 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [43]:
def balance(acct, budgets=True):
    lst = DB.select(Entry, account=acct)
    if budgets:
        res = sum(e.value for e in lst)
    else:
        res = sum(e.value for e in lst if not Tag.B in e.tags)
    return res

In [44]:
balance('expenses:food', False)

40.0

In [45]:
%timeit balance('expenses:food', False)

The slowest run took 11.01 times longer than the fastest. This could mean that an intermediate result is being cached.
1.88 ms ± 1.77 ms per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [46]:
def balance(acct, budgets=True):
    lst = DB.select(Entry, account=acct)
    res = sum(e.value for e in lst)
    if not budgets:
        res -= sum(e.value for e in lst if Tag.B in e.tags)
    return res

In [47]:
balance('expenses:food', False)

40.0

In [48]:
%timeit balance('expenses:food', False)

544 μs ± 108 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
